In [123]:
#in anaconda commandprompt:
#conda install pytorch torchvision cpuonly -c pytorch

#pip install jupyterlab ultralytics
# or
#pip install ultralytics

#pip install ultralytics==8.2.76

# you can train your own model in yolo and use it as well

## Base detection and tracker

In [36]:
#Video_Path = r'./videos/video360p.mp4'
from ultralytics import YOLO
import cv2
from IPython.display import clear_output

model= YOLO('yolov8n.pt')
Video_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\Python-Yolo8-object-Detection\videos\video1111.mp4"

In [38]:
def basedetectionandtracker(Path):
    cap = cv2.VideoCapture(Path)
    ret = True
    while ret:
        ret, frame = cap.read()
        clear_output(wait=True)
        if ret:
            
            # detect objects
            # track object
            results = model.track(frame, persist=True)
        
            # plot results    
            frame_ = results[0].plot()
        
            # visualize
            cv2.imshow('frame', frame_)
            if cv2.waitKey(1) & 0xff == ord('q'):
                break

In [14]:
basedetectionandtracker(Video_Path)


0: 384x640 10 persons, 47.5ms
Speed: 2.0ms preprocess, 47.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


## Detect and Track any object in YOLO8 from file and webcam

In [42]:
import cv2
import torch
import cv2
from ultralytics import YOLO
from IPython.display import clear_output

# Function to process video with optimizations
def process_video(video_path, frame_skip=1, ConfidenceThreshold = 0.75, ShowVideo = False, Resolution = (0, 0)):
    # Load video
    model= YOLO('yolov8n.pt')
    cap = cv2.VideoCapture(video_path)


    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        clear_output(wait=True)
        if not ret:
            break
         # Skip frames if needed
        if frame_count % frame_skip != 0:
             frame_count += 1
             continue

        # Optionally resize the frame for faster processing
        if Resolution != (0,0):
            frame = cv2.resize(frame, Resolution)   

        
        # Use the model to track objects in the frame
        results = model.track(frame, show=False, persist=True, conf=ConfidenceThreshold)
        if ShowVideo:
            annotated_frame = results[0].plot()
            cv2.imshow("FRAME",annotated_frame)
            del annotated_frame

        del frame 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()
# Example usage
# Load a lighter YOLO model (e.g., YOLOv8-tiny) and set the video path



In [28]:
Video_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\object tracker\LowQualityVideos\Original videos\video2.mp4"
test = process_video(Video_Path, frame_skip=2, ConfidenceThreshold = 0.25, ShowVideo = True, Resolution = (0, 0))


0: 544x640 1 person, 59.5ms
Speed: 2.0ms preprocess, 59.5ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)


## Read from webcame and detect objects

In [4]:
Video_Path = 1 # it take a bit time
test = process_video(Video_Path, frame_skip=2, ConfidenceThreshold = 0.25, ShowVideo = True, Resolution = (1080, 720))


0: 448x640 2 persons, 69.7ms
Speed: 4.0ms preprocess, 69.7ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 85.6ms
Speed: 15.0ms preprocess, 85.6ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 61.1ms
Speed: 3.0ms preprocess, 61.1ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 57.5ms
Speed: 3.0ms preprocess, 57.5ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 66.5ms
Speed: 1.0ms preprocess, 66.5ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 62.5ms
Speed: 3.0ms preprocess, 62.5ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 55.8ms
Speed: 2.0ms preprocess, 55.8ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 53.5ms
Speed: 2.0ms preprocess, 53.5ms inference, 1.0ms postprocess per image at shape 

KeyboardInterrupt: 

## Detect and Track any object in YOLO8 from stream Url

In [84]:
import cv2
from ultralytics import YOLO
from vidgear.gears import CamGear   
from IPython.display import clear_output


# Function to process video with optimizations
def process_live_video(video_url, frame_skip=1, ConfidenceThreshold = 0.75, ShowVideo = False, Resolution = (0, 0)):
    stream = CamGear(source=video_url, stream_mode = True, logging=False).start()
    # Load video
    model= YOLO('yolov8n.pt')

    frame_count = 0
    while True:
        frame = stream.read()
        clear_output(wait=True)
        if frame is None:
            break
        clear_output(wait=True)

        if frame_count % frame_skip != 0:
             frame_count += 1
             continue
            
        # Optionally resize the frame for faster processing
        if Resolution != (0,0):
            frame = cv2.resize(frame, Resolution)   

        results = model.track(frame, show=False, persist=True, conf=ConfidenceThreshold)
        if ShowVideo:
            annotated_frame = results[0].plot()
            cv2.imshow("FRAME",annotated_frame)
            del annotated_frame

        del frame 
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

        frame_count += 1
    cap.release()
    cv2.destroyAllWindows()
# Example usage
# Load a lighter YOLO model (e.g., YOLOv8-tiny) and set the video path


In [87]:
Video_Path_from_youtube = r"https://www.youtube.com/watch?v=jMVYZ9uMkwI"
test = process_live_video(Video_Path_from_youtube, frame_skip=3, ConfidenceThreshold = 0.25 ,ShowVideo = True)


0: 384x640 1 person, 1 car, 68.4ms
Speed: 3.0ms preprocess, 68.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


KeyboardInterrupt: 

## Extract and data result from YOLO8 from

In [318]:
import cv2
import pandas as pd
from ultralytics import YOLO
from IPython.display import clear_output
from datetime import datetime, timedelta

# Function to process video with optimizations
def process_video_GetResults(video_path, frame_skip=1, ConfidenceThreshold = 0.75, ShowVideo = False, Resolution = (0, 0), excel_file='predictions.xlsx'):
    
    # Load video
    model= YOLO('yolov8n.pt')
    cap = cv2.VideoCapture(video_path)

    all_detections = []  # To store all detections

    
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        #clear_output(wait=True)
        if not ret:
            break
         # Skip frames if needed
        if frame_count % frame_skip != 0:
             frame_count += 1
             continue

        # Optionally resize the frame for faster processing
        if Resolution != (0,0):
            frame = cv2.resize(frame, Resolution)   
        current_time = datetime.now()    
        
        # Use the model to track objects in the frame
        results = model.track(frame, show=False, persist=True, conf=ConfidenceThreshold)
        print('=================================================================')

        detections = extract_detections(results, current_time, frame_count, model.names)
        all_detections.extend(detections)  # Add detections to the main list
       
        print('=================================================================')
        
        if ShowVideo:
            annotated_frame = results[0].plot()
            cv2.imshow("FRAME",annotated_frame)
            del annotated_frame

        del frame 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()
    
    df = pd.DataFrame(all_detections)

    # Append to an existing Excel file or create a new one
    with pd.ExcelWriter(excel_file, mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Detections')
# Example usage
# Load a lighter YOLO model (e.g., YOLOv8-tiny) and set the video path



In [320]:
def extract_detections(results,timestamp, frame_number, class_names):
    detections = []

    # Loop over each result to extract details
    for r in results:
        for det in r.boxes:
            # Get class index and map it to class name
            class_index = int(det.cls[0])
            class_name = class_names[class_index]
            
            detection = {
                'Time': timestamp.strftime('%Y-%m-%d %H:%M:%S'),  # Format time as string
                'FrameNumber': frame_number,   # Use frame number as timestamp; can be modified to use actual time
                'Object': class_name,  # Class name (e.g., person, car)
                'ID': int(det.id[0]) if det.id is not None else None,  # Unique ID for tracking
                'Confidence': float(det.conf[0])  # Confidence score
            }
            detections.append(detection)

    return detections

In [ ]:
Video_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\Python-Yolo8-object-Detection\video360p.mp4"
Save_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\Python-Yolo8-object-Detection\predictions.xlsx"
test = process_video_GetResults(Video_Path, frame_skip=60, ConfidenceThreshold = 0.50, ShowVideo = True, Resolution = (0, 0), excel_file = Save_Path)

# Save each 1 second

In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
from IPython.display import clear_output
from datetime import datetime, timedelta

# Function to process video with optimizations
def process_video_GetResults(video_path, frame_skip=1, ConfidenceThreshold = 0.75, ShowVideo = False, Resolution = (0, 0), Save_file='predictions.xlsx'):
    
    # Load video
    model= YOLO('yolov8n.pt')
    cap = cv2.VideoCapture(video_path)

    all_detections = []  # To store all detections

    start_time = datetime.now()
    last_save_time = start_time  # Track last save time
    
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        #clear_output(wait=True)
        if not ret:
            break
         # Skip frames if needed
        if frame_count % frame_skip != 0:
             frame_count += 1
             continue

        # Optionally resize the frame for faster processing
        if Resolution != (0,0):
            frame = cv2.resize(frame, Resolution)   
        current_time = datetime.now()    
        
        # Use the model to track objects in the frame
        results = model.track(frame, show=False, persist=True, conf=ConfidenceThreshold)
        print('=================================================================')


        current_time = datetime.now()

        if (current_time - last_save_time).total_seconds() >= save_interval:
            detections = extract_detections(results, current_time, frame_count, model.names)
            all_detections.extend(detections)  # Add detections to the main list
            df = pd.DataFrame(all_detections)
            if not os.path.exists(excel_file):
                df.to_excel(excel_file, index=False, sheet_name='Detections')
            else:
                with pd.ExcelWriter(excel_file, mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:
                    df.to_excel(writer, index=False, sheet_name='Detections')
            last_save_time = current_time
            all_detections = []  # Clear the list after saving

        print('=================================================================')
        
        if ShowVideo:
            annotated_frame = results[0].plot()
            cv2.imshow("FRAME",annotated_frame)
            del annotated_frame

        del frame 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()
    
    df = pd.DataFrame(all_detections)

    print(Save_file)
    # Append to an existing Excel file or create a new one
    with pd.ExcelWriter(Save_file, mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Detections')
# Example usage
# Load a lighter YOLO model (e.g., YOLOv8-tiny) and set the video path



In [ ]:
def extract_detections(results,timestamp, frame_number, class_names):
    detections = []

    # Loop over each result to extract details
    for r in results:
        for det in r.boxes:
            # Get class index and map it to class name
            class_index = int(det.cls[0])
            class_name = class_names[class_index]
            
            detection = {
                'Time': timestamp.strftime('%Y-%m-%d %H:%M:%S'),  # Format time as string
                'FrameNumber': frame_number,   # Use frame number as timestamp; can be modified to use actual time
                'Object': class_name,  # Class name (e.g., person, car)
                'ID': int(det.id[0]) if det.id is not None else None,  # Unique ID for tracking
                'Confidence': float(det.conf[0])  # Confidence score
            }
            detections.append(detection)

    return detections

In [ ]:
Video_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\Python-Yolo8-object-Detection\video360p.mp4"
Save_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\Python-Yolo8-object-Detection\predictions.xlsx"
test = process_video_GetResults(Video_Path, frame_skip=60, ConfidenceThreshold = 0.50, ShowVideo = True, Resolution = (0, 0), excel_file = Save_Path)